인공지능과 기계학습 최종 작성본

#제목 : 딥러닝을 활용한 전력데이터 예측실험

#3조 - electronic 

* 조장: 황미림

* 조원: 홍준호, 노경우


* 최초 작업일 2020.05.31

* 최종 수정일 2020.06.21


* 함수구현자 표기 이외의 코드는 공동으로 제작하였습니다. 





In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
from tensorflow import keras
print(keras.__version__)

2.3.0-tf


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import seaborn as sns
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RNN
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
#from sklearn.preprocessing import LabelEncoder #문자열로 된 컬럼이 있을 때
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

In [ ]:
dataset = read_csv("C:/Users/ty009/문서/ITRC/ITRC_1F_GROUP_12.csv", header=0, index_col=0)
values = dataset.values

In [ ]:
groups = [0,1,2,3]
i = 1
plt.figure(figsize=(15, 12), dpi= 60, facecolor='w', edgecolor='k')
for group in groups:
    plt.subplot(len(groups),1,i)
    plt.plot(values[:,group])
    plt.title(dataset.columns[group], y=0, loc='right', fontsize=15)
    i += 1
plt.show()

<br><br>
## 3. 다변량 예측 모델
####  데이터 준비
<br>
series_to_supervised() : 시계열 데이터셋을 supervised learning 문제로 변경하기 위한 함수

* data : 목록 또는 2D NumPy 배열 형식의 관측치<br>

* n_in : (입력값) 지연 관찰 수로, 값은 1에서 len(data) 사잇값. 기본값은 1<br>

* n_out : 출력(y)되는 관측치의 수인 것 같음. 값은 0에서 len(data)-1 사잇값. 기본값은 1. 선택 사항임<br>

* dropnan : NaN 값인 행을 삭제할지 여부를 나타내는 부울 값. 기본값은 True이며, 역시 선택사항<br>

* return : 지도학습을 위해 프래임 된 pandas DataFrame

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    sns.heatmap(df.isnull(), cbar=False)
    plt.show()
    df[3] = df[3].fillna(method = 'pad')
    sns.heatmap(df.isnull(), cbar=False)
    plt.show()
    
    ## input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' %(j+1, i)) for j in range(n_vars)]
        
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
            
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
        
    return agg

In [ ]:
# 코드 구현자: 홍준호 , 노경우 
reframed = series_to_supervised(values, 5, 1, False)
reframed = reframed.drop(reframed.index[0:5])
for i in range(5):
    reframed = reframed.drop(reframed.columns[0+i:3+i], axis='columns')  #여기 숫자는 고정!
print(reframed.head())
print("values ", len(values), "reframed ", len(reframed))

<br>데이터를 모두 float(소수) 형식으로 변경하여 0에서 1 사이 값으로 만들고 위에서 만든 함수를 적용시킴<br><br>

문자열로 구성된 컬럼이 있어 예측을 위해 숫자 형식으로 변환할 필요가 있을 때 아래의 코드의 처음에 다음을 추가한다.<br>
encoder = LabelEncoder() <br>
values[:,1] = encoder.fit_transform(values[:,1])<br>

* 위의 코드와 위치 변경됨

In [ ]:
# ensure all data is float
reframed = reframed.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(reframed)

In [ ]:
print(scaled)

<br><br>
#### 모델 정의 및 맞춤

준비된 dataset을 train data와 validation data, test data로 분할함<br>
데이터 정제 과정에서 12시~13시를 제거했으므로 하루를 23시로 계산함

In [ ]:
values = scaled
n_train = 14 * 23 * 60
n_validation = n_train + 7 * 23 * 60
train = values[:n_train, :]
validation = values[n_train:n_validation, :]
test = values[n_validation:, :]

train data와 test data의 입력값(X)과 출력값(Y)을 설정함<br>
* var1(Active Power)은 Y에, 나머지는 모두 X에 넣음

In [ ]:
train_X, train_Y = train[:, :-1], train[:, -1]
validation_X, validation_Y = validation[:, :-1], validation[:, -1]
test_X, test_Y = test[:, :-1], test[:, -1]

In [ ]:
print(train_X.shape, train_Y.shape, validation_X.shape, validation_Y.shape, test_X.shape, test_Y.shape)

입력 내용을 3D로 변경 [samples, timesteps, features]<br>

* [samples, timesteps, features]가 층, 행, 열일 거라고 생각하고 있음<br>

다음을 실행하면 train data와 validation data, test data가 인쇄된다.

In [ ]:
# 함수 구현자: 홍준호 , 노경우 
train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])
validation_X = validation_X.reshape(validation_X.shape[0], 1, validation_X.shape[1])
test_X = test_X.reshape(test_X.shape[0], 1, test_X.shape[1])
print(train_X.shape, train_Y.shape, validation_X.shape, validation_Y.shape)
print("train_X")
print(train_X)

모델 디자인<br>
Active Power를 예측하기 위해 첫번째 hidden layer와 output layer에 뉴런을 정의할 것<br>
* LSTM(): lstm 모델 생성, 입력값은 각각 timestep과 feature<br>
* Dense() : 예측하고자 하는 target의 개수<br>     첫번째 값 : 노드의 개수(원하는 숫자로 하되, 너무 적으면 학습이 잘 되지 않으니 5 이상의 정수로 한다),<br> input_shape : 입력값의 모양. (1,)일 때는 한 개씩 여러번 해야 하기 때문에 정하지 않았다는 의미
* compile : 모델을 어떻게 학습할지 정하는 곳
* 연속적인 예측을 하는 것이기 때문에 loss function은 mean_squared_error, optimizer는 adam을 자주 사용한다고 함<br>
* compile 기본 형식 : compile(optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None) https://keras.io/ko/models/sequential/

In [ ]:
# 함수 구현자: 홍준호 , 노경우 

j = 0
slListStr = ["SimpleRNN", "LSTM", "GRU"]

for i in [SimpleRNN, LSTM, GRU]:
    globals()["%s_slModel" %slListStr[j]] = Sequential()
    globals()["%s_slModel" %slListStr[j]].add(i(30, activation='softmax', input_shape=(train_X.shape[1], train_X.shape[2])))
    globals()["%s_slModel" %slListStr[j]].add(Dropout(0.2))
    globals()["%s_slModel" %slListStr[j]].add(Dense(30, activation='elu'))
    globals()["%s_slModel" %slListStr[j]].add(Dropout(0.2))
    globals()["%s_slModel" %slListStr[j]].add(Dense(1, activation='sigmoid'))
    globals()["%s_slModel" %slListStr[j]].compile(loss='mse', metric=['accuracy'], optimizer='adam')
    globals()["%s_slModel" %slListStr[j]].summary()
    j += 1
    print("\n\n")

In [ ]:
# 함수 구현자: 홍준호 , 노경우 

#history = model.fit(train_X, train_Y, epochs=20, batch_size=72, verbose=2)
j = 0
for i in [SimpleRNN, LSTM, GRU]:
    print("\n\nStudy %s" %slListStr[j])
    early_stopping = EarlyStopping(monitor='val_loss', patience=15)
    globals()["%s_history" %slListStr[j]] = globals()["%s_slModel" %slListStr[j]].fit(train_X, train_Y, epochs=150, batch_size=50, validation_data=(validation_X, validation_Y), verbose=1, callbacks=[early_stopping])
    j += 1

plot 그리기

In [ ]:
# 함수 구현자: 황미림 

fig, ax = plt.subplots(1, 3, figsize=(20, 5))

ax[0].plot(SimpleRNN_history.history['loss'], label='train')
ax[0].plot(SimpleRNN_history.history['val_loss'], label='validation')
ax[0].title.set_text('1 Layer SimpleRNN')
ax[0].legend()

ax[1].plot(LSTM_history.history['loss'], label='train')
ax[1].plot(LSTM_history.history['val_loss'], label='validation')
ax[1].title.set_text('1 Layer LSTM')
ax[1].legend()

ax[2].plot(GRU_history.history['loss'], label='train')
ax[2].plot(GRU_history.history['val_loss'], label='validation')
ax[2].title.set_text('1 Layer GRU')
ax[2].legend()

plt.show()

<br><br>
## 4. vaildation data 예측&정확도 측정
#### validation 예측 생성

In [ ]:
SimpleRNN_vali_yhat = SimpleRNN_slModel.predict(validation_X)
LSTM_vali_yhat = LSTM_slModel.predict(validation_X)
GRU_vali_yhat = GRU_slModel.predict(validation_X)

#### test 예측 생성

In [ ]:
SimpleRNN_test_yhat = SimpleRNN_slModel.predict(test_X)
LSTM_test_yhat = LSTM_slModel.predict(test_X)
GRU_test_yhat = GRU_slModel.predict(test_X)

In [ ]:
validation_X = validation_X.reshape((validation_X.shape[0], validation_X.shape[2]))
print(validation_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
print(test_X)

In [ ]:
validation_y = np.array(validation_Y).reshape(validation_Y.shape[0],1)
print(validation_y)
test_y = np.array(test_Y).reshape(test_Y.shape[0],1)
print(test_y)

#### 예측값 Plot

In [ ]:
# 함수 구현자: 황미림
# 수정본 작성: 홍준호 , 노경우 

fig, ax = plt.subplots(3, 2, figsize=(20, 15), dpi= 60, facecolor='w', edgecolor='k')

# Simple RNN
ax[0][0].plot(validation_y, label='origin')
ax[0][0].plot(SimpleRNN_vali_yhat, label='predict')
ax[0][0].title.set_text('1 Layer SimpleRNN Validation')
ax[0][0].legend(loc='upper right')

ax[0][1].plot(test_y, label='origin')
ax[0][1].plot(SimpleRNN_test_yhat, label='predict')
ax[0][1].title.set_text('1 Layer SimpleRNN Test')
ax[0][1].legend(loc='upper right')


# LSTM
ax[1][0].plot(validation_y, label='origin')
ax[1][0].plot(LSTM_vali_yhat, label='predict')
ax[1][0].title.set_text('1 Layer LSTM Validation')
ax[1][0].legend(loc='upper right')

ax[1][1].plot(test_y, label='origin')
ax[1][1].plot(LSTM_test_yhat, label='predict')
ax[1][1].title.set_text('1 Layer LSTM Test')
ax[1][1].legend(loc='upper right')


# GRU
ax[2][0].plot(validation_y, label='origin')
ax[2][0].plot(GRU_vali_yhat, label='predict')
ax[2][0].title.set_text('1 Layer GRU Validation')
ax[2][0].legend(loc='upper right')

ax[2][1].plot(test_y, label='origin')
ax[2][1].plot(GRU_test_yhat, label='predict')
ax[2][1].title.set_text('1 Layer GRU Test')
ax[2][1].legend(loc='upper right')

plt.show();

#### 평가

In [ ]:
# 함수 구현자: 홍준호 , 노경우 

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_log_error

for i in ["SimpleRNN", "LSTM", "GRU"]:
    print("1 Layer %s Model Validation Evaluation\n" %i)
    MAE_vail =  mean_absolute_error(validation_y, globals()["%s_vali_yhat" %i])
    MSE_vail = mean_squared_error(validation_y, globals()["%s_vali_yhat" %i])
    RMSE_vail = np.sqrt(MSE_vail)
    MSLE_vail = mean_squared_log_error(validation_y, globals()["%s_vali_yhat" %i])
    RMSLE_vail = np.sqrt(mean_squared_log_error(validation_y, globals()["%s_vali_yhat" %i]))
    R2_vail = r2_score(validation_y, globals()["%s_vali_yhat" %i])
    MAPE_vail = np.mean(np.abs((globals()["%s_vali_yhat" %i] - validation_y) / globals()["%s_vali_yhat" %i])) * 100
    
    MAE_test =  mean_absolute_error(test_y, globals()["%s_test_yhat" %i])
    MSE_test = mean_squared_error(test_y, globals()["%s_test_yhat" %i])
    RMSE_test = np.sqrt(MSE_test)
    MSLE_test = mean_squared_log_error(test_y, globals()["%s_test_yhat" %i])
    RMSLE_test = np.sqrt(mean_squared_log_error(test_y, globals()["%s_test_yhat" %i]))
    R2_test = r2_score(test_y, globals()["%s_test_yhat" %i])
    MAPE_test = np.mean(np.abs((globals()["%s_test_yhat" %i] - test_y) / globals()["%s_test_yhat" %i])) * 100
    
    print("\t<Validation>\t<Test>")
    print("MAE\t%.10f"%MAE_vail, "\t%.10f"%MAE_test)
    print("MSE\t%.10f"%MSE_vail, "\t%.10f"%MSE_test)
    print("RMSE\t%.10f"%RMSE_vail, "\t%.10f"%RMSE_test)
    print("MSLE\t%.10f"%MSLE_vail, "\t%.10f"%MSLE_test)
    print("RMSLE\t%.10f"%RMSLE_vail, "\t%.10f"%RMSLE_test)
    print("MAPE\t%.10f"%MAPE_vail, "\t%.10f"%MAPE_test)
    print("R2\t%.10f"%R2_vail, "\t%.10f"%R2_test)
    print("\n\n")